# Selenium
## Web Browser automation and Scraping

Selenium is a Python package that allows you to automate your web browser and to scrape data off web pages.

###### Download Link and Instructions: http://selenium-python.readthedocs.io/installation.html

# Example 1: Scraping from IMDB

In [1]:
# imports

import numpy as np
import pandas as pd
from selenium import webdriver
import time # this is for sleeping

In [2]:
# initializing the browser and going to a web page

# open Firefox
browser = webdriver.Firefox()

# go to the web page that we want to scrape from
browser.get('http://www.imdb.com/title/tt1490017/')

# wait for browser/page to load before doing anything else
'''
If you don't do this, selenium may get confused while running 
the next command because whatever object it looks for may not yet be there.
So when running a command that will open a new web page it is usually
a good idea to sleep for a few seconds.
''' 
time.sleep(2)

Selenium allows you to select web page elements in a variety of ways:
                                    1. .find_element_by_class_name
                                    2. .find_element_by_css_selector
                                    3. .find_element_by_id
                                    4. .find_element_by_link_text
                                    5. .find_element_by_name
                                    6. .find_element_by_partial_link_text
                                    7. .find_element_by_tag_name
                                    8. .find_element_by_xpath

In [3]:
# Select the table of cast members using "inspect element" in your browser
table = browser.find_element_by_class_name("cast_list").get_attribute('outerHTML')

# use pd.read_html() to read the table in python
cast = pd.read_html(table)
# returns as a list of one object………
print(type(cast))
print(len(cast))
# ……… that object being, convenienthy, a pandas data frame!
print(type(cast[0]))

cast_df = cast[0]
cast_df

<class 'list'>
1
<class 'pandas.core.frame.DataFrame'>


,0,1,2,3
0,"Cast overview, first billed only:",NaN,NaN,NaN
1,NaN,Will Arnett,...,Batman / Bruce Wayne (voice)
2,NaN,Elizabeth Banks,...,Wyldstyle / Lucy (voice)
3,NaN,Craig Berry,...,Blake / Additional Voices (voice)
4,NaN,Alison Brie,...,Unikitty (voice)
5,NaN,David Burrows,...,Octan Robot / Additional Voices (voice)
6,NaN,Anthony Daniels,...,C-3PO (voice)
7,NaN,Charlie Day,...,Benny (voice)
8,NaN,Amanda Farinos,...,Mom (voice)
9,NaN,Keith Ferguson,...,Han Solo (voice)


In [4]:
# close the browser now that we've got the data we need
'''
just for convenience sake because Selenium opens up a new 
iteration of Firefox everytime you run it which can get annoying
'''
browser.close()

In [15]:
# a little cleaning to make our data frame pretty…

# drop the columns with useless information
cast_df = cast_df.drop([0,2], 1)

# drop the first row of NaN's
cast_df = cast_df.drop(cast_df.index[0])

# name the columns something descriptive
cast_df.columns = ['Actor', 'Role']

# voila!
cast_df

,Actor,Role
1,Will Arnett,Batman / Bruce Wayne (voice)
2,Elizabeth Banks,Wyldstyle / Lucy (voice)
3,Craig Berry,Blake / Additional Voices (voice)
4,Alison Brie,Unikitty (voice)
5,David Burrows,Octan Robot / Additional Voices (voice)
6,Anthony Daniels,C-3PO (voice)
7,Charlie Day,Benny (voice)
8,Amanda Farinos,Mom (voice)
9,Keith Ferguson,Han Solo (voice)
10,Will Ferrell,Lord Business (voice) / President Business (vo...


# Example 2: Slack Bot

In [5]:
# first, let's pick a channel and a message to send to that channel

# pick your channel
channel = 'bot_world'
# write a message
message = 'Hi there Slack!'

In [6]:
# open Firefox
browser = webdriver.Firefox()

# go to the web page that we want to scrape from
browser.get('http://slack.com/signin')

# wait for browser/page to load before doing anything else
time.sleep(2)

In [7]:
# LOGIN

# type the slack team name
browser.find_element_by_id("domain").send_keys("hackcville")

# press continue button
browser.find_element_by_id("submit_team_domain").click()

# wait for next page to load
time.sleep(2)

In [8]:
# supply username and password for slack
email = input("Email: ")
password = input("Password: ")

# type username
browser.find_element_by_id("email").send_keys(email)
# type password
browser.find_element_by_id("password").send_keys(password)
# click sign in button
browser.find_element_by_id("signin_btn").click()

# wait for the next page to load
time.sleep(15)

Email: ct3fb@virginia.edu
Password: Whisper


In [9]:
# navigate to the channel you want to write in

'''
This finds the elements with class name = "overflow_ellipse" 
whithin the element with id = "channels" and puts them all in a list.
'''
channels = browser.find_element_by_id('channels').find_elements_by_class_name('overflow_ellipsis')

# lets see what the inner HTML elements look like
channels[0].get_attribute('innerHTML')

'\n\t\t\t\t\t\tannouncements\n\t\t\t\t'

In [10]:
# that is ugly ^^^
# let's clean it up with .strip() to clear the whitespace in the for loop

'''
this just iterates through the stripped inner HTML of the items 
in the 'channels' list until it finds the one we want then clicks it
'''
for i in range(len(channels)):
    if (channels[i].get_attribute('innerHTML').strip() == channel):
        # click on the 'bot_world' channel
        channels[i].click()
        
        # break out of the for loop
        break

In [11]:
# find the element for the text bar
text_bar = browser.find_element_by_class_name('ql-editor.ql-blank').find_element_by_css_selector('p')
# type your message
text_bar.send_keys(message)
# press enter
text_bar.send_keys(u'\ue007')

#you can repeat this cell with different messages to send multiple messages to the channel

So if you ever want to write a message to Slack but you're in class and don't want to get called out for not working, just whip out this script and no one will suspect anything.

## Challenge
1. Scrape the entire table by making Selenium press the "See full cast >>" button at the bottom of the table which opens up the full cast list, not just the first 15 members.

2. Scrape the full cast list from multiple imdb pages by using the search bar to navigate between the pages.